In [24]:
from model import BodyposeModel
import cv2
import numpy as np
import math
import time
from scipy.ndimage.filters import gaussian_filter
import matplotlib.pyplot as plt
import matplotlib
import torch
from torchvision import transforms
import util
from torchsummary import summary
from body import Body
import import_ipynb
from train_model import BodyposeBackbone, MainModel
from coco_dataset import COCODataset, transform_image
from copy import deepcopy
import random
import pandas as pd
from tqdm import tqdm
import sys

/tmp/ipykernel_37641/1145751693.py:6: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


In [25]:
NO_OF_OUTPUTS = 17

In [26]:
colors = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0], [85, 255, 0], [0, 255, 0], \
            [0, 255, 85], [0, 255, 170], [0, 255, 255], [0, 170, 255], [0, 85, 255], [0, 0, 255], [85, 0, 255], \
            [170, 0, 255], [255, 0, 255], [255, 0, 170], [255, 0, 85], [255, 255, 255]]

In [27]:
new_order = [0, 15, 14, 17, 16, 5, 2, 6, 3, 7, 4, 11, 8, 12, 9, 13, 10]

In [28]:
backbone_model_path = '../model/body_pose_model.pth'

In [29]:
backbone_model = BodyposeBackbone()

In [30]:
backbone_params = torch.load(backbone_model_path)

In [31]:
backbone_dict = util.transfer(backbone_model, backbone_params)

In [32]:
backbone_model.load_state_dict(backbone_dict)

<All keys matched successfully>

In [33]:
main_model = MainModel(NO_OF_OUTPUTS)

Create dataset from COCO .json file

In [34]:
train_data = COCODataset('/media/jakub/One Touch/coco_pose/coco2017labels-pose/coco-pose', transforms=transform_image(split='train'))

loading annotations into memory...
Done (t=0.21s)
creating index...
index created!


In [35]:
skeleton = train_data.coco.cats[1]['skeleton']
skeleton = [[connection[0]-1, connection[1]-1] for connection in skeleton]

In [36]:
index = random.randint(0, 100)
anns = []
while anns == []:
    for id in train_data.coco.getAnnIds():
        if train_data.coco.anns[id]['image_id'] == train_data.coco.getImgIds()[index]:
            ann = train_data.coco.anns[id]
            if 1 in ann['keypoints'] or 2 in ann['keypoints']:
                anns.append(ann)
    if anns == []:
        index = random.randint(0, 100)



In [37]:
bboxes = [ann['bbox'] for ann in anns]

In [38]:
bboxes = [[int(element) for element in box] for box in bboxes]

In [39]:
keypoints = [ann['keypoints'] for ann in anns]

In [40]:
new_keypoints = []
for keypoint in keypoints:
    new_keypoints.append([[keypoint[i], keypoint[i+1], keypoint[i+2]] for i in range(0, len(keypoint), 3)])
keypoints = new_keypoints

In [41]:
image = train_data._load_image(train_data.coco.getImgIds()[index])
for bbox in bboxes:
    cv2.rectangle(image, (bbox[0], bbox[1]), (bbox[0]+bbox[2], bbox[1]+bbox[3]), (0, 0, 255))
for i, connection in enumerate(skeleton):
    for keypoint in keypoints:
        point1 = keypoint[connection[0]]
        point2 = keypoint[connection[1]]
        if point1[2] != 0 and point2[2] != 0:
            cv2.line(image, point1[:-1], point2[:-1], color=colors[i], thickness=3)
cv2.imshow('Example image', image)
cv2.waitKey(5_000)
cv2.destroyAllWindows()

In [42]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [43]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True, num_workers=4, collate_fn=collate_fn)

In [44]:
def train(backbone_model, main_model, loss, optimizer, inputs, outputs):
    optimizer.zero_grad()
    main_model.train()

    out1 = backbone_model(inputs)
    logits = main_model(out1)
    output = loss.forward(logits, outputs)
    output.backward()
    optimizer.step()

    return output.item()

In [45]:
loss = torch.nn.MSELoss()
params = [p for p in main_model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params)

In [48]:
device = torch.device("cuda")

In [51]:
def train_one_epoch(backbone_model, main_model, optimizer, loader, epoch, device):
    # model.to(device)
    main_model.train()

#     lr_scheduler = None
#     if epoch == 0:
#         warmup_factor = 1.0 / 1000 # do lr warmup
#         warmup_iters = min(1000, len(loader) - 1)

#         lr_scheduler = optim.lr_scheduler.LinearLR(optimizer, start_factor = warmup_factor, total_iters=warmup_iters)

    all_losses = []
    all_losses_dict = []

    for images, targets in tqdm(loader):
        # images = list(image.to(device) for image in images)
        # targets = [{k: torch.tensor(v).to(device) for k, v in t.items()} for t in targets]

        out1 = backbone_model(images)
        logits = main_model(out1)
        loss_dict = loss.forward(logits, targets)

        # loss_dict = model(images, targets) # the model computes the loss automatically if we pass in targets
        losses = sum(loss for loss in loss_dict.values())
        loss_dict_append = {k: v.item() for k, v in loss_dict.items()}
        loss_value = losses.item()

        all_losses.append(loss_value)
        all_losses_dict.append(loss_dict_append)

        if not math.isfinite(loss_value):
            print(f"Loss is {loss_value}, stopping trainig") # train if loss becomes infinity
            print(loss_dict)
            sys.exit(1)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

#         if lr_scheduler is not None:
#             lr_scheduler.step() #

    all_losses_dict = pd.DataFrame(all_losses_dict) # for printing
    print("Epoch {}, lr: {:.6f}, loss: {:.6f}, loss_classifier: {:.6f}, loss_box: {:.6f}, loss_rpn_box: {:.6f}, loss_object: {:.6f}".format(
        epoch, optimizer.param_groups[0]['lr'], np.mean(all_losses),
        all_losses_dict['loss_classifier'].mean(),
        all_losses_dict['loss_box_reg'].mean(),
        all_losses_dict['loss_rpn_box_reg'].mean(),
        all_losses_dict['loss_objectness'].mean()
    ))

In [52]:
num_epochs=4

for epoch in range(num_epochs):
    train_one_epoch(backbone_model, main_model, optimizer, train_loader, epoch, device)
        # output = train(backbone_model, main_model, loss, optimizer, inputs, outputs)

  0%|          | 0/674 [00:01<?, ?it/s]


TypeError: conv2d() received an invalid combination of arguments - got (tuple, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!tuple of (Tensor, Tensor, Tensor, Tensor)!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!tuple of (Tensor, Tensor, Tensor, Tensor)!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
